So for part 2, we should also find the weight that each disk supports. So we'll want some new structures, let's use `dict`s, which implement the tree structures, and the weight distributions.

Let's start off by creating a tree that represents just the structure of the tower; we can worry about the weights after:

In [1]:
import re

from collections import Counter

Start with the example input again:

In [2]:
input_str='''pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)'''

Let's go through each line: if it contains balancing information, add that to a set `supported_set`, and add any other cases to a set `disks_set`.

This time, rather than simply adding to a set `supported_set`, let's use a dict which implements the tree structure of the tower. I'm also changing the disks representation, so that we store the weight, and eventually, the total supported weight:

In [3]:
disks_dict={}

for nextLine_str in input_str.split('\n'):
    if '->' in nextLine_str:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\) ->(?P<rest>.*)', nextLine_str)
        
        disks_dict[m_re.group('name')]={'weight':int(m_re.group('weight')),
                                        'supporting':{disk.strip() for disk in m_re.group('rest').split(',')}}
    else:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\)', nextLine_str)
        
        disks_dict[m_re.group('name')]={'weight':int(m_re.group('weight'))}
print(disks_dict)


{'pbga': {'weight': 66}, 'xhth': {'weight': 57}, 'ebii': {'weight': 61}, 'havc': {'weight': 66}, 'ktlj': {'weight': 57}, 'fwft': {'weight': 72, 'supporting': {'cntj', 'xhth', 'ktlj'}}, 'qoyq': {'weight': 66}, 'padx': {'weight': 45, 'supporting': {'pbga', 'havc', 'qoyq'}}, 'tknk': {'weight': 41, 'supporting': {'padx', 'ugml', 'fwft'}}, 'jptl': {'weight': 61}, 'ugml': {'weight': 68, 'supporting': {'gyxo', 'jptl', 'ebii'}}, 'gyxo': {'weight': 61}, 'cntj': {'weight': 57}}


And now find the disk which isn't in one of the `supporting` sets:

In [4]:
nonRoot_set=set()
for vals in disks_dict.values():
    nonRoot_set=nonRoot_set.union(vals.get('supporting', {}))
nonRoot_set

rootDisk_str=(set(disk for disk in disks_dict)-nonRoot_set).pop()
rootDisk_str

'tknk'

Now build the total weight (own weight + supported weights) recursively:

In [5]:
def set_total_weights(startDisk_str, disksIn_dict):
    '''
    Find the total weight supported (including itself) by each
    disk in the tower. Update disksIn_ls with those weights.
    
    Returns the total weight on startDisk_str
    '''
    
    # If the disk is not supporting any other disks:
    if 'supporting' not in disksIn_dict[startDisk_str]:
        # Set the totalWeight to the disk's own weight, and
        # return that value
        disksIn_dict[startDisk_str]['totalWeight']=disksIn_dict[startDisk_str]['weight']
        return disksIn_dict[startDisk_str]['totalWeight']
    
    # Otherwise, build the total weights from the start
    # disk recursively:
    else:
        disksIn_dict[startDisk_str]['totalWeight'] = \
            sum([set_total_weights(d, disksIn_dict) 
                 for d in disksIn_dict[startDisk_str]['supporting']]) + \
            disksIn_dict[startDisk_str]['weight']
        return disksIn_dict[startDisk_str]['totalWeight']
        

print(set_total_weights('tknk', disks_dict))
print()
disks_dict


778



{'cntj': {'totalWeight': 57, 'weight': 57},
 'ebii': {'totalWeight': 61, 'weight': 61},
 'fwft': {'supporting': {'cntj', 'ktlj', 'xhth'},
  'totalWeight': 243,
  'weight': 72},
 'gyxo': {'totalWeight': 61, 'weight': 61},
 'havc': {'totalWeight': 66, 'weight': 66},
 'jptl': {'totalWeight': 61, 'weight': 61},
 'ktlj': {'totalWeight': 57, 'weight': 57},
 'padx': {'supporting': {'havc', 'pbga', 'qoyq'},
  'totalWeight': 243,
  'weight': 45},
 'pbga': {'totalWeight': 66, 'weight': 66},
 'qoyq': {'totalWeight': 66, 'weight': 66},
 'tknk': {'supporting': {'fwft', 'padx', 'ugml'},
  'totalWeight': 778,
  'weight': 41},
 'ugml': {'supporting': {'ebii', 'gyxo', 'jptl'},
  'totalWeight': 251,
  'weight': 68},
 'xhth': {'totalWeight': 57, 'weight': 57}}

So we seem to be building the tower correctly. Now we need to work out which is the incorrect weight.

Actually, we don't need to be too bothered about finding precisly which is the incorrect disk: we only need to find out which is the odd one out, and return the weight of the others.

In [6]:
def odd_one_out(dictIn_dict):
    '''
    Take a dictionary of key:value pairs. Return the
    triple (name, totalWeight, otherWeight) of the odd one 
    out, where otherWeight is the weight of the other members
    of the list. False if there isn't one.
    '''

    # Return False if all keys have same value    
    if len(set(dictIn_dict.values()))==1:
        return False

    # Raise an error if more than two values:
    assert len(set(dictIn_dict.values()))<=2, "too many values"

    # Raise an error if not a single exception:
    cc=sorted(Counter(dictIn_dict.values()).values())
    assert cc[0]==1 and cc[1]>1, "too many values"
    
    # Otherwise, find the odd one out and return the triple:
    keys_ls=list(dictIn_dict)
    
    # Get the values of the first two keys:
    v0=dictIn_dict[keys_ls[0]]
    v1=dictIn_dict[keys_ls[1]]
    
    # If they're different, get the next value:
    if v0!=v1:
        v2=dictIn_dict[keys_ls[2]]
        # ... and return the appropriate triple
        if v0==v2:
            return (keys_ls[1], v1, v0)
        else:
            return (keys_ls[0], v0, v1)
    else:
        # Otherwise, loop until find the exception...
        for key in keys_ls:
            if dictIn_dict[key]!=v0:
                # ... and return the triple
                return (key, dictIn_dict[key], v0)

dictIn_dict={'a':2, 'b':3, 'c':3, 'd':3}
odd_one_out(dictIn_dict)

('a', 2, 3)

OK. Those are the preliminaries. Now we should just be able to run through the disk dictionary and make the adjustment when we find the odd one out:

In [7]:
def find_incorrect_disk(startDisk_str, disksIn_dict, weightDiff_i=0):
    '''Return the disk with the change that needs to be made to its weight'''
    # Get the weights on the current disk:
    weightsOnCurrent_dict={disk:disks_dict[disk]['totalWeight'] 
                           for disk in disksIn_dict[startDisk_str]['supporting']}
    # If the weights on the current disk are equal, return
    # the current disk:
    if len(set(weightsOnCurrent_dict.values()))==1:
        return (startDisk_str, weightDiff_i)
    
    # otherwise, apply the function to the odd one out:
    (d, w1, w2)=odd_one_out(weightsOnCurrent_dict)
    return find_incorrect_disk(d, disksIn_dict, w2-w1)
    
find_incorrect_disk('tknk', disks_dict)
    

('ugml', -8)

Actually, to solve it, we need to add the final argument returned by `find_incorrect_disk` to the weight of the disk in question:

In [8]:
(d, c)=find_incorrect_disk('tknk', disks_dict)

disks_dict[d]['weight']+c

60

OK, that's the right answer, although I can't help thinking I might have made rather heavy weather of that one. Still, let's do it with my input:

In [9]:
with open('data/day7.txt') as fIn:
    input_str=fIn.read()

In [10]:
disks_dict={}

for nextLine_str in input_str.split('\n'):
    if '->' in nextLine_str:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\) ->(?P<rest>.*)', nextLine_str)
        
        disks_dict[m_re.group('name')]={'weight':int(m_re.group('weight')),
                                        'supporting':{disk.strip() for disk in m_re.group('rest').split(',')}}
    else:
        m_re=re.match('\s*(?P<name>\w+) \((?P<weight>\d+)\)', nextLine_str)
        if m_re:
            disks_dict[m_re.group('name')]={'weight':int(m_re.group('weight'))}
disks_dict


{'nzyiue': {'weight': 57},
 'pdmkag': {'weight': 39},
 'bogbg': {'weight': 13},
 'nubay': {'weight': 45},
 'dukzh': {'weight': 17},
 'kpjxln': {'supporting': {'dzzbvkv', 'gzdxgvj', 'jidxg', 'wsocb'},
  'weight': 44},
 'cxjyxl': {'weight': 83},
 'vusplt': {'supporting': {'mcfst', 'orrwx'}, 'weight': 151},
 'mxrfq': {'weight': 98},
 'bdoez': {'weight': 62},
 'vrajpg': {'weight': 78},
 'qzsowpu': {'weight': 90},
 'nrxoha': {'weight': 51},
 'xtjrkv': {'supporting': {'jlbhafs', 'pyocxtt'}, 'weight': 351},
 'rlnii': {'supporting': {'ljvpv', 'wkumzkr', 'xjosf'}, 'weight': 18986},
 'ddrrgp': {'weight': 23},
 'wladmn': {'weight': 42},
 'ryskzh': {'supporting': {'wsyya', 'xbqpjo'}, 'weight': 209},
 'jbjkp': {'weight': 34},
 'fnfiur': {'weight': 86},
 'zxzkl': {'weight': 82},
 'hbmxey': {'supporting': {'khkuxc', 'miauii', 'mjuwde', 'tztycfl'},
  'weight': 1869},
 'zjyqcfa': {'weight': 97},
 'zbtck': {'weight': 48},
 'mgsasl': {'weight': 39},
 'cemygp': {'weight': 34},
 'iolrkmv': {'supporting': {

In [11]:
nonRoot_set=set()
for vals in disks_dict.values():
    nonRoot_set=nonRoot_set.union(vals.get('supporting', {}))
nonRoot_set

rootDisk_str=(set(disk for disk in disks_dict)-nonRoot_set).pop()
rootDisk_str

'hlhomy'

In [12]:
set_total_weights(rootDisk_str, disks_dict)

781448

In [13]:
(d, c)=find_incorrect_disk(rootDisk_str, disks_dict)
d

'apjxafk'

In [14]:
disks_dict[d]['weight']+c

1505